In [1]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import os

import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import random
from scipy.stats import randint



# Load your dataset
# Load the dataset
file_path = 'D:\\1 code AI'
file_names = ['new_data_v1.csv']
datasets = [pd.read_csv(os.path.join(file_path, file)) for file in file_names]
dataset = pd.concat(datasets, ignore_index=True)

# Count the number of labels 0 and 1
label_counts = dataset['Label'].value_counts()
print("Label counts:")
print(label_counts)

dataset.info()

from sklearn.model_selection import train_test_split

# Check if columns exist before dropping
columns_drop = ["Label"]
existing_columns_to_drop = [col for col in columns_drop if col in dataset.columns]

X = dataset.drop(columns=existing_columns_to_drop)
y = dataset["Label"]

print(X.head())
print(X.info())
print(y.head())
print(y.value_counts())


# Split the data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

print("Training set label distribution:")
print(y_train.value_counts())
print("Test set label distribution:")
print(y_test.value_counts())




Label counts:
Label
0    204368
1     34767
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239135 entries, 0 to 239134
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   row     239135 non-null  int64  
 1   col     239135 non-null  int64  
 2   year    239135 non-null  int64  
 3   month   239135 non-null  int64  
 4   day     239135 non-null  int64  
 5   hour    239135 non-null  int64  
 6   B04B    239135 non-null  float64
 7   B05B    239135 non-null  float64
 8   B06B    239135 non-null  float64
 9   B09B    239135 non-null  float64
 10  B10B    239135 non-null  float64
 11  B11B    239135 non-null  float64
 12  B12B    239135 non-null  float64
 13  B14B    239135 non-null  float64
 14  B16B    239135 non-null  float64
 15  I2B     239135 non-null  float64
 16  I4B     239135 non-null  float64
 17  IRB     239135 non-null  float64
 18  VSB     239135 non-null  float64
 19  WVB     239135 n

In [3]:
#random search
param_dist = {
    "n_estimators": randint(850, 1050),  
    'max_features': [None, 'sqrt', 'log2'],     
    'max_depth': randint(40,60),        
    'min_samples_split': randint(8, 13),   
    'min_samples_leaf': randint(2, 6),    
    'class_weight': ['balanced', 'balanced_subsample']  
}




rf = RandomForestClassifier(random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rf_random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                       n_iter=20, cv=skf, scoring='f1', n_jobs=-1, verbose=10, random_state=42)

rf_random_search.fit(X_train, y_train)

print("Best parameters found: ", rf_random_search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters found:  {'class_weight': 'balanced', 'max_depth': 59, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 921}


In [4]:
best_rf = rf_random_search.best_estimator_

y_pred = best_rf.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96     40874
           1       0.73      0.76      0.75      6953

    accuracy                           0.92     47827
   macro avg       0.85      0.86      0.85     47827
weighted avg       0.93      0.92      0.93     47827

